### Import statements

In [23]:
# GENERAL GUIDLINES
# 20 different categories
# each category has around 1000 docs.
# train a classfier 
# remove words with freq 1, 2
# don't add stop words in dictonary
# sort dictonary

from os import listdir
from os.path import isfile, join
import string
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import math as ma

#a simple helper function to convert a 2D array to 1D, without using numpy
def flatten(list):
    new_list = []
    for i in list:
        for j in i:
            new_list.append(j)
    return new_list

### Get data in terms of x and y

In [2]:
def get_x_y(my_path):
    folders = []

    for f in listdir(my_path):
        if f != '.DS_Store':
            folders.append(f) 
            
    #---------
    
    files = []
    for folder_name in folders:
        folder_path = join(my_path, folder_name)
        files.append([f for f in listdir(folder_path)])
        
    #----------
    
    x = []
    y = []
    for fo in range(len(folders)):
        for fi in files[fo]:
            y.append(folders[fo])
            x.append(join(my_path, join(folders[fo], fi)))
    
    return x, y

x, y = get_x_y("20_newsgroups")

### Get Train and Test data

In [3]:
doc_x_train, doc_x_test, doc_y_train, doc_y_test = train_test_split(x, y, random_state=0, test_size=0.25)
len(doc_x_train)

14997

### Preporcess remove stop words and preprocess words

In [4]:
stopwords=np.loadtxt('https://a5a5687e-a-62cb3a1a-s-sites.googlegroups.com/site/kevinbouge/stopwords-lists/stopwords_en.txt?attachauth=ANoY7crcABAfBg6e5ZA_6lZ79_r4NuPw2j4KdOLBq9sySv5C18RCykRhnGoN-pG2hfvk4B2jqk2lDxK5Al2FVEfUXu4dbWV-7HDgYA7JrFSMyvIu7r9BNi1uAPe_wiuEFCLptEBoK3D3NOBv-un9McQ2nVZ2RsPF1EQmwLN3n7wS6OlSWMUeI6xaXvU_Tvsqwi5DQOOaIx6zw11Ry6wptN3n3aUuF8Tw8wJYhWoPQCpKTZ0eukRtmvk%3D&attredirects=0&d=1',dtype= str , delimiter=',').tolist()

def remove_stopwords(words):
    words = [word for word in words if not word in stopwords]
    return words

def preprocess(words):
    #we'll make use of python's translate function,that maps one set of characters to another
    #we create an empty mapping table, the third argument allows us to list all of the characters we want to delete
    
    #first we will try to filter out some  unnecessary data like tabs, punctuation
    
    # the character: ' appears in a lot of stopwords and changes meaning of words if removed
    #hence it is removed from the list of symbols that are to be discarded from the documents
    
    p = (string.punctuation).replace("'", "")  + "\t"
    table = str.maketrans('', '', p)
    words = [word.translate(table) for word in words]
    
    
    #some white spaces may be added to the list of words, due to the translate function & nature of our documents
    #we remove them below
    words = [str for str in words if str]
        
    #we will also remove just-numeric strings as they do not have any significant meaning in text classification
    words = [word for word in words if not word.isdigit()]
    
    #after removal of so many characters it may happen that some strings have become blank, we remove those
    words = [str for str in words if str]
    
    #we also normalize the cases of our words
    words = [word.lower() for word in words]
    
    #we try to remove words with only 2 characters or 1 character
    words = [word for word in words if len(word) > 2]
    
    return words



def tokenize_sentence(line):
    words = line.strip().split(" ")
    words = preprocess(words)
    words = remove_stopwords(words)
    return words


#function to convert a document into list of words

def tokenize(path):
    #load document as a list of lines
    f = open(path, 'r', encoding="utf8", errors='ignore')
    text_lines = f.readlines()
    
    #initiazing an array to hold all the words in a document
    doc_words = []
    
    #traverse over all the lines and tokenize each one with the help of helper function: tokenize_sentence
    for line in text_lines:
        doc_words.append(tokenize_sentence(line))

    return doc_words

### Get train data in freq format

In [5]:
list_of_train_words_per_doc = []

for document in doc_x_train:
    list_of_train_words_per_doc.append(flatten(tokenize(document)))
    
list_of_words = np.array(flatten(list_of_train_words_per_doc))


words, counts = np.unique(list_of_words, return_counts=True)
freq, wrds = (list(i) for i in zip(*(sorted(zip(counts, words), reverse=True))))

n = 2000
features = wrds[0:n]


dictionary_train = {}
doc_num = 1
for doc_words in list_of_train_words_per_doc:
    np_doc_words = np.array(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary_train[doc_num] = dict(zip(w,c))
    doc_num = doc_num + 1

#now we make a 2D array having the frequency of each word of our feature set in each individual documents

x_train = []
for k in dictionary_train.keys():
    row = []
    for f in features:
        if(f in dictionary_train[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary_train[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    x_train.append(row)
    
x_train = np.array(x_train)
y_train = np.array(doc_y_train)

### Get test data in freq format

In [6]:
list_of_test_words_per_doc = []

for document in doc_x_test:
        list_of_test_words_per_doc.append(flatten(tokenize(document)))
        

dictionary_test = {}
doc_num = 1
for doc_words in list_of_test_words_per_doc:
    #print(doc_words)
    np_doc_words = np.array(doc_words)
    w, c = np.unique(np_doc_words, return_counts=True)
    dictionary_test[doc_num] = dict(zip(w,c))
    doc_num = doc_num + 1
    

#now we make a 2D array having the frequency of each word of our feature set in each individual documents

x_test = []
for k in dictionary_test.keys():
    row = []
    for f in features:
        if(f in dictionary_test[k].keys()):
            #if word f is present in the dictionary of the document as a key, its value is copied
            #this gives us no. of occurences
            row.append(dictionary_test[k][f]) 
        else:
            #if not present, the no. of occurences is zero
            row.append(0)
    x_test.append(row)
    

x_test = np.array(x_test)
y_test = np.array(doc_y_test)
    

### SKlearn Classifier. Accuracy/Score - 0.86

In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)

y_predict = clf.predict(x_test)

clf.score(x_test, y_test)

0.86

In [34]:
print(classification_report(y_test,y_predict))                 # Print classification report

                          precision    recall  f1-score   support

             alt.atheism       0.73      0.80      0.76       240
           comp.graphics       0.80      0.77      0.78       244
 comp.os.ms-windows.misc       0.84      0.82      0.83       240
comp.sys.ibm.pc.hardware       0.87      0.84      0.85       256
   comp.sys.mac.hardware       0.87      0.92      0.90       249
          comp.windows.x       0.92      0.86      0.89       233
            misc.forsale       0.79      0.90      0.84       259
               rec.autos       0.88      0.92      0.90       253
         rec.motorcycles       0.89      0.96      0.92       231
      rec.sport.baseball       0.97      0.98      0.97       236
        rec.sport.hockey       0.97      0.98      0.98       261
               sci.crypt       0.96      0.92      0.94       269
         sci.electronics       0.83      0.88      0.85       246
                 sci.med       0.96      0.88      0.92       284
         

### My implementation

In [20]:
def fit(X_train, Y_train):                                          # This function is used to fit training data into our model
    result = {}                                                     # This dictionary is going to be useful in later calculations
    class_values = set(Y_train) 
    for current_class in class_values:                              # We create keys for all the possible classes
        result[current_class] = {}                                  # We create a dictionary as value for each key itself
        current_class_rows = (Y_train == current_class)             # Obtain rows for the current class
        X_train_current = X_train[current_class_rows]               # Filter the x_train for current class
        Y_train_current = Y_train[current_class_rows]               # Filter the y_train for current class
        num_features = X_train.shape[1]                             
        result[current_class]["count"] = len(Y_train_current)       # It gives total number of features of our data
        a=0                                                         # To get total number of a particular feature 
        for j in range(num_features):                               # traverse each feature
            result[current_class][j]=X_train_current[:,j].sum()     # Get total number of current feature
            a+=result[current_class][j]                             # Increment a by total number of current feature
        result[current_class]['total']=a                            # Assign a, will be used in later calculations
    result["total"] = len(Y_train)                                  # It gives total element present in our dictionary
    return result                                                   # Return result



def probability(dictionary, x, current_class): 
    output = ma.log(dictionary[current_class]["total"]) - ma.log(dictionary["total"])
    num_features = len(dictionary[current_class].keys()) - 2
    for j in range( num_features ):
        if x[j]==0:
            continue
        count_current_class_with_value_j = dictionary[current_class][j]+1
        count_current_class = dictionary[current_class]['total']+num_features
        current_j_probablity = ma.log(count_current_class_with_value_j) - ma.log(count_current_class)
        output = output + current_j_probablity
    return output

def doSinglePrediction(x,dictionary):                                  # Function to predict class for a single row
    classes = dictionary.keys()                                        # Get all possible classes
    best_p = -100                                                      # Initialise best probablity & class to some -ve number
    best_class = -100 
    first_run = True                                                   # Running for first time = True
    for current_class in classes:                                      # Iterate over each possible class
        if (current_class == "total"):                                 # Ignore 'total' key
            continue
        p_current_class = probability(dictionary, x, current_class)    # Get probablity for x belonging to current class
        if (first_run or p_current_class > best_p):                    # If this is greatest probablity till now, change the
            best_p = p_current_class                                    # value of greatest probability & predicted class
            best_class = current_class
        first_run = False                                              # First run complete
    return best_class                                                  # Return the predicted class out of all classes


def predict(x_test, d):
    y_pred = []
    for x in x_test:
        x_class = doSinglePrediction(x, dictionary)
        y_pred.append(x_class)
    return y_pred

In [15]:
dictionary = fit(x_train, y_train)

In [26]:
Y_pred = predict(x_test, dictionary)

In [29]:
accuracy_score(y_test, Y_pred)

0.8682

In [31]:
print(classification_report(y_test,Y_pred))

                          precision    recall  f1-score   support

             alt.atheism       0.75      0.81      0.78       240
           comp.graphics       0.82      0.84      0.83       244
 comp.os.ms-windows.misc       0.90      0.82      0.86       240
comp.sys.ibm.pc.hardware       0.92      0.86      0.89       256
   comp.sys.mac.hardware       0.89      0.92      0.90       249
          comp.windows.x       0.94      0.85      0.89       233
            misc.forsale       0.78      0.91      0.84       259
               rec.autos       0.87      0.93      0.90       253
         rec.motorcycles       0.88      0.94      0.91       231
      rec.sport.baseball       0.96      0.99      0.97       236
        rec.sport.hockey       0.99      0.97      0.98       261
               sci.crypt       0.98      0.93      0.95       269
         sci.electronics       0.81      0.90      0.85       246
                 sci.med       0.96      0.87      0.91       284
         

In [ ]:
# # features = ['cat','bat', 'dog', 'lot', 'fol', 'tol', 'dol']
# # x = np.array([
# #     [1,2,3,2,4,5,6],
# #     [0,2,3,2,4,5,6], #b
# #     [1,4,5,2,4,5,6], #b
# #     [0,2,2,2,4,5,0],
# #     [1,2,3,2,4,5,0], #b
# #     [1,2,3,4,3,5,6],
# #     [1,2,3,2,4,5,6] #b
# # ])

# # x_test =  np.array([
# #     [1,2,3,4,3,5,6],
# #     [1,2,3,4,3,5,6]
# # ])

# # y = np.array(['a','b','b','a','b','a','b'])
# def fit(X_train, Y_train):
#     result = {}
#     class_values = set(Y_train)
#     for current_class in class_values:
#         result[current_class] = {}
#         result["total_data"] = len(Y_train)
        
#         current_class_rows = (Y_train == current_class)
        
#         X_train_current = X_train[current_class_rows]
#         Y_train_current = Y_train[current_class_rows]
#         num_features = X_train.shape[1]
#         result[current_class]["total_class_count"] =  X_train_current.shape[0]
#         result[current_class]["total_no_of_words"] =  X_train_current.sum()
#         for j in range(0, num_features):
#             f = features[j]
#             result[current_class][f] = X_train_current[:,j].sum()  
#     return result

# def probability(dictionary, x, current_class):
#     output = np.log(dictionary[current_class]["total_class_count"]) - np.log(dictionary["total_data"])
#     for f in features:
#         count_current_class_with_value_xj = dictionary[current_class][f] + 1
#         count_current_class = dictionary[current_class]["total_no_of_words"]+ len(features)
#         current_xj_probablity = np.log(count_current_class_with_value_xj) - np.log(count_current_class)
#         output = output + current_xj_probablity
#     return output

# def predictSinglePoint(dictionary, x):
#     classes = dictionary.keys()
#     best_p = -1000
#     best_class = -1
#     first_run = True
#     for current_class in classes:
#         if (current_class == "total_data"):
#             continue
#         p_current_class = probability(dictionary, x, current_class)
#         if (first_run or p_current_class > best_p):
#             best_p = p_current_class
#             best_class = current_class
#         first_run = False
#     return best_class

# def predict(dictionary, x_test):
#     y_pred = []
#     for x in x_test:
#         x_class = predictSinglePoint(dictionary, x)
#         y_pred.append(x_class)
#     return y_pred

# dictionary = fit(x_train, y_train)
# Y_pred = predict(dictionary, x_test[0:100])
# accuracy_score(y_test[0:100], Y_pred)